<a href="https://colab.research.google.com/github/arimbawa/Aplikasi-IoT/blob/main/Minggu_3_Integrasi_IoT_ke_Flask_dengan_HTTP_%26_MQTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Materi Minggu 3: Integrasi IoT ke Flask dengan HTTP & MQTT**

**Tujuan Pembelajaran:**







*   Mahasiswa memahami bagaimana perangkat IoT (sensor) mengirimkan data ke server.
*   Mahasiswa mampu mengirimkan data sensor melalui HTTP POST dan MQTT.
*   Mahasiswa dapat membuat Flask server untuk menerima data.



# Langkah Praktikum

1.   Setup Flask Server (Google Colab)



In [ ]:
!pip install flask pyngrok
!pip install paho-mqtt

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import paho.mqtt.client as mqtt
import json
import threading

# Replace "YOUR_NGROK_AUTH_TOKEN" with your actual ngrok authtoken
ngrok.set_auth_token("xxxxxxxxxxxxx")

# Buat app Flask
app = Flask(__name__)
sensor_data = []

# MQTT Settings
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
MQTT_TOPIC = "unram/iot/sensor"

# MQTT Callbacks
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT Broker!")
        client.subscribe(MQTT_TOPIC)
        print(f"Subscribed to topic: {MQTT_TOPIC}")
    else:
        print(f"Failed to connect, return code {rc}\n")

def on_message(client, userdata, msg):
    print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")
    try:
        data = json.loads(msg.payload.decode())
        # Assuming the MQTT message contains 'light' data
        if 'light' in data:
            sensor_data.append({"light": data['light']})
            print("Light data added to sensor_data:", data['light'])
        else:
            print("MQTT message does not contain 'light' data.")
    except json.JSONDecodeError:
        print("Failed to decode JSON from MQTT message.")

# Create MQTT Client
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Connect to MQTT Broker in a separate thread
def connect_mqtt():
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_forever()

mqtt_thread = threading.Thread(target=connect_mqtt)
mqtt_thread.start()


@app.route('/sensor', methods=['POST'])
def receive_sensor():
    data = request.json
    sensor_data.append(data)
    print("Received data via HTTP:", data)
    return jsonify({"status": "success", "received": data})

@app.route('/data', methods=['GET'])
def get_data():
    return jsonify(sensor_data)

# Buka tunnel ngrok di port 5000
public_url = ngrok.connect(5000).public_url
print("Flask server URL:", public_url)

# Jalankan Flask
if __name__ == '__main__':
    app.run(port=5000)

To use `pyngrok`, you need to get an authtoken from the ngrok dashboard ([https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)). Once you have it, you can add it to the code using `ngrok.set_auth_token()`.

Setelah dijalankan, akan muncul URL publik https://xxxxx.ngrok.io → endpoint ini akan dipakai oleh client (HTTP/MQTT).

2. Simulasi Sensor & Kirim Data via HTTP, jalankan di Note terpisah.

In [ ]:
import requests, random, time

FLASK_URL = "https://87869cee103a.ngrok-free.app/sensor"  # ganti dengan URL ngrok Flask

for i in range(5):  # kirim 5 kali data
    temp = round(random.uniform(20, 30), 2)
    hum = round(random.uniform(40, 60), 2)
    data = {"temperature": temp, "humidity": hum}

    r = requests.post(FLASK_URL, json=data)
    print(f"Sent {data}, Response: {r.json()}")
    time.sleep(2)


3. Kirim Data via MQTT (HiveMQ Broker), jalankan di Note terpisah.

In [ ]:
!pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt
import json, random, time

MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
MQTT_TOPIC = "unram/iot/sensor"

client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    print("Connected to MQTT Broker:", rc)

def on_publish(client, userdata, mid):
    print("Message Published:", mid)

client.on_connect = on_connect
client.on_publish = on_publish
client.connect(MQTT_BROKER, MQTT_PORT, 60)
client.loop_start()

for i in range(50):
    light = random.randint(0, 1023) # Simulate light sensor data (e.g., from 0 to 1023)
    payload = {"light": light}
    client.publish(MQTT_TOPIC, json.dumps(payload))
    print("Sent:", payload)
    time.sleep(2)

client.loop_stop()
client.disconnect()

4. Flask Subscriber (Opsional, jika ingin menerima MQTT langsung di server Flask)

Untuk integrasi penuh, Flask bisa bertindak sebagai subscriber MQTT dengan library paho-mqtt lalu menyimpan data ke endpoint /data.

# Output yang Diharapkan

Data sensor random terkirim via HTTP ke Flask → dapat dicek di GET /data.

Data sensor terkirim via MQTT ke broker HiveMQ (bisa dicek di HiveMQ Web Client
, subscribe ke topic unram/iot/sensor).